# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
! pip install cassandra-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.8 MB/s eta 0:00:00


In [2]:
# Import Python packages 
import datetime
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/content
/content/event_data
['/content/event_data/2018-11-19-events.csv', '/content/event_data/2018-11-01-events.csv', '/content/event_data/2018-11-23-events.csv', '/content/event_data/2018-11-30-events.csv', '/content/event_data/2018-11-06-events.csv', '/content/event_data/2018-11-22-events.csv', '/content/event_data/2018-11-04-events.csv', '/content/event_data/2018-11-03-events.csv', '/content/event_data/2018-11-02-events.csv', '/content/event_data/2018-11-17-events.csv', '/content/event_data/2018-11-28-events.csv', '/content/event_data/2018-11-13-events.csv', '/content/event_data/2018-11-09-events.csv', '/content/event_data/2018-11-11-events.csv', '/content/event_data/2018-11-20-events.csv', '/content/event_data/2018-11-29-events.csv', '/content/event_data/2018-11-18-events.csv', '/content/event_data/2018-11-08-events.csv', '/content/event_data/2018-11-24-events.csv', '/content/event_data/2018-11-25-events.csv', '/content/event_data/2018-11-10-events.csv', '/content/event_data/2018

## Pandas Method - Processing the files to create the data file csv that will be used for Apache Casssandra tables 

In [40]:
# making a time stamp
now = datetime.datetime.now()
time_stamp = now.strftime("%c").replace(" ", "")
time_stamp = time_stamp.replace(":", "")

# for every filepath in the file path list 
combined_frame = pd.DataFrame()
for path in file_path_list:

# reading csv file 
    data = pd.read_csv(path,encoding = 'utf8')
    combined_frame = pd.concat([combined_frame, data], ignore_index=True)   
 # extracting each csv one by one and append it        
    #data.to_csv(f"{time_stamp}.csv", mode='a', index=False, encoding = 'utf8')

# saving the combined frame
combined_frame.to_csv(f"{time_stamp}.csv", mode='w', index=False, encoding = 'utf8')
combined_csv = f"{time_stamp}.csv"
print(combined_csv)

WedJan110158492023.csv


### Clean DATA , DROP unconstant Data, DROP Unused DATA

In [41]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format
data_frame = pd.read_csv(combined_csv)

#### Explore the Data & Check for Consistency

In [42]:
# How many rows and columns does df_apps have? What are the column names?
# What does the data look like?
# Look at a random sample of 5 different rows with
print(data_frame.shape)
print(data_frame.columns)
print(data_frame.sample())

(8056, 17)
Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userId'],
      dtype='object')
              artist       auth firstName gender  itemInSession lastName  \
2568  Tapes n Tapes  Logged In      Kate      F             13  Harrell   

      length level                  location method      page  \
2568  191.63  paid  Lansing-East Lansing, MI    PUT  NextSong   

             registration  sessionId                song  status  \
2568 1,540,470,000,000.00        537  Say Back Something     200   

                       ts  userId  
2568 1,542,130,000,000.00   97.00  


#### Dropping the columns that are not relevant to our queries  

In [43]:
# drop the cols that are not needed
data_frame.drop(["auth", "method", "page", "registration", "status", "ts"], axis=1, inplace=True)
print(data_frame.head())

                  artist firstName gender  itemInSession lastName  length  \
0                    NaN      Lily      F              0    Burns     NaN   
1  Explosions In The Sky    Adelyn      F              0   Jordan  497.48   
2                    NaN    Adelyn      F              1   Jordan     NaN   
3                    NaN       NaN    NaN              2      NaN     NaN   
4                    NaN       NaN    NaN              3      NaN     NaN   

  level                               location  sessionId               song  \
0  free  New York-Newark-Jersey City, NY-NJ-PA        689                NaN   
1  free     Chicago-Naperville-Elgin, IL-IN-WI        458  Your Hand In Mine   
2  free     Chicago-Naperville-Elgin, IL-IN-WI        458                NaN   
3  free                                    NaN        458                NaN   
4  free                                    NaN        458                NaN   

   userId  
0   32.00  
1    7.00  
2    7.00  
3     Na

#### Cleaning the data here

In [44]:
# Remove nun & dublicate values
print(f'Missing values for data set?: {data_frame.isna().values.any()}')
print(f'Missing values for data set?: {data_frame.isna().values.sum()}')
data_frame.dropna(inplace=True)
print(f'Missing values for data set?: {data_frame.isna().values.any()}')
print(f'Missing values for data set?: {data_frame.isna().values.sum()}')
# Remove duplicates
print(f'duplicates values for data set?: {data_frame.duplicated().values.any()}')
print(f'duplicates values for data set?: {data_frame.duplicated().values.sum()}')
print(data_frame.duplicated)  


Missing values for data set?: True
Missing values for data set?: 5138
Missing values for data set?: False
Missing values for data set?: 0
duplicates values for data set?: False
duplicates values for data set?: 0
<bound method DataFrame.duplicated of                                     artist firstName gender  itemInSession  \
1                    Explosions In The Sky    Adelyn      F              0   
6     Paul Van Dyk Featuring Jessica Sutta     Layla      F              0   
7                               Tim Hughes     Layla      F              1   
8     David Cassidy & The Partridge Family     Layla      F              2   
9                              Snow Patrol     Layla      F              3   
...                                    ...       ...    ...            ...   
8050                        The Cat Empire      Lily      F            104   
8052                          STRATOVARIUS      Lily      F              1   
8053                           The Mantles      

#### check our data after  cleanning 

In [45]:
print(data_frame.shape)
print(data_frame.columns)
print(data_frame.sample())

(6820, 11)
Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')
              artist firstName gender  itemInSession lastName  length level  \
5988  Massive Attack      Lily      F             57     Koch  316.50  paid   

                                location  sessionId       song  userId  
5988  Chicago-Naperville-Elgin, IL-IN-WI        764  Karmacoma   15.00  


#### Change Columns type to the desired format

In [46]:
print(data_frame.describe())
print(data_frame.info())

       itemInSession   length  sessionId   userId
count       6,820.00 6,820.00   6,820.00 6,820.00
mean           22.76   247.03     599.18    54.68
std            23.44   102.98     284.95    28.16
min             0.00    15.86       3.00     2.00
25%             4.00   197.32     374.00    29.00
50%            15.00   232.97     605.00    49.00
75%            35.00   274.12     834.00    80.00
max           126.00 2,594.87   1,114.00   101.00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 1 to 8055
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   firstName      6820 non-null   object 
 2   gender         6820 non-null   object 
 3   itemInSession  6820 non-null   int64  
 4   lastName       6820 non-null   object 
 5   length         6820 non-null   float64
 6   level          6820 non-null   object 
 7   location       6820 non-null   object 
 8 

In [54]:
data_frame["itemInSession"] = pd.to_numeric(data_frame["itemInSession"])
data_frame["length"] = pd.to_numeric(data_frame["length"])
data_frame["sessionId"] = pd.to_numeric(data_frame["sessionId"])
data_frame["userId"] = pd.to_numeric(data_frame["userId"],downcast='integer' )


#### See the new data types and if the data is ready

In [51]:
print(data_frame.describe())
print(data_frame.info())

       itemInSession   length  sessionId   userId
count       6,820.00 6,820.00   6,820.00 6,820.00
mean           22.76   247.03     599.18    54.68
std            23.44   102.98     284.95    28.16
min             0.00    15.86       3.00     2.00
25%             4.00   197.32     374.00    29.00
50%            15.00   232.97     605.00    49.00
75%            35.00   274.12     834.00    80.00
max           126.00 2,594.87   1,114.00   101.00
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 1 to 8055
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   firstName      6820 non-null   object 
 2   gender         6820 non-null   object 
 3   itemInSession  6820 non-null   int64  
 4   lastName       6820 non-null   object 
 5   length         6820 non-null   float64
 6   level          6820 non-null   object 
 7   location       6820 non-null   object 
 8 

## Method 2 - Processing the files to create the data file csv that will be used for Apache Casssandra tables 

In [ ]:
'''
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
'''

In [ ]:
'''
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
    
'''

# Part II. Model & Denormalize the DB for the Needed Queries , Optimizing for Fast, Responsive Reads

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4


                    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "<ENTER INSERT STATEMENT HERE>"
        query = query + "<ASSIGN VALUES HERE>"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[#], line[#]))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


                    

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()